# Examples for the usage of the BUBE app through the REST API with Python

## Settings and Imports

In [ ]:
import os

import requests

In [ ]:
# Path of the running BUBE APP
BUBE_URL = "http://localhost:8000"

## Sending Images as binary data

In [ ]:
mime_types = {"jpg": "image/jpeg", "jpeg": "image/jpeg", "png": "image/png", "heif": "image/heif"}

In [ ]:
def embed_images_binary(folder_path: str):
    image_files = []

    for filename in os.listdir(folder_path):
        if filename.lower().endswith((".png", ".jpg", ".jpeg", ".heif")):
            file_path = os.path.join(folder_path, filename)
            mime_type = mime_types.get(filename.split(".")[-1].lower(), "image/jpeg")
            image_files.append(("images", (filename, open(file_path, "rb"), mime_type)))

    if not image_files:
        print("No Images found in the folder")

    try:
        response = requests.post(f"{BUBE_URL}/embeddings", files=image_files)
        response.raise_for_status()
        print("Images successfully uploaded.")
        res = response.json()
    except requests.exceptions.RequestException:
        print(f"Request failed. Is the BUBE App running on {BUBE_URL}?")
        res = None
    finally:
        for _, file_tuple in image_files:
            file_tuple[1].close()

    return res

In [ ]:
def save_images_feex_binary(folder_path: str):
    image_files = []

    for filename in os.listdir(folder_path):
        if filename.lower().endswith((".png", ".jpg", ".jpeg", ".heif")):
            file_path = os.path.join(folder_path, filename)
            mime_type = mime_types.get(filename.split(".")[-1].lower(), "image/jpeg")
            image_files.append(("images", (filename, open(file_path, "rb"), mime_type)))

    if not image_files:
        print("No Images with specified filenames could be found.")
        return

    try:
        response = requests.post(f"{BUBE_URL}/feex/insert", files=image_files)
        response.raise_for_status()
        print("Images successfully uploaded.")
    except requests.exceptions.RequestException:
        print(f"Request failed. Is the BUBE App running on {BUBE_URL}?")
    finally:
        # close file handlers
        for _, file_tuple in image_files:
            file_tuple[1].close()

In [ ]:
def duplicate_check_single_image_binary(image_folder_path: str, img_filename: str):
    img_filepath = os.path.join(image_folder_path, img_filename)
    files = [("images", (img_filename, open(img_filepath, "rb"), "image/jpeg"))]
    try:
        response = requests.post(f"{BUBE_URL}/feex", files=files)
        response.raise_for_status()
        # print(f"Duplicate check for {img_filepath} successful.")
        res = response.json()
    except requests.exceptions.RequestException:
        print(f"Request failed. Is the BUBE App running on {BUBE_URL}?")
        res = None
    finally:
        for _, file_tuple in files:
            file_tuple[1].close()
    return res